In [1]:
from utils import r14, r32, r50, r125, r134a, r143a, r170, atom_type, opt_atom_types
import numpy as np
import unyt as u
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real

2024-02-28 14:35:10.679601: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 14:35:10.725318: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 14:35:10.725374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 14:35:10.727208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 14:35:10.735668: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 14:35:10.736287: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
#Load class properies for each molecule
r14_class = r14.R14Constants()
r32_class = r32.R32Constants()
r50_class = r50.R50Constants()
r125_class = r125.R125Constants()
r134a_class = r134a.R134aConstants()
r143a_class = r143a.R143aConstants()
r170_class = r170.R170Constants()

In [3]:
repeats = 1
seed = 1
save_data = False
at_class = atom_type.AT_Scheme_7()

In [4]:
molec = "R32"
molec_data_dict = {molec:r32_class} #, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
all_gp_dict = {molec : all_gp_dict[molec]}
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, save_data)
ls_results = at_optimizer.optimize_ats()
ls_results

20 841.6365362161803
20 144.00538537855664
20 38.57299808907601
20 23.43806370946847
20 14.793477764589552
20 10.003894207747129
20 7.026680938873629
20 5.109304226990056
20 3.818707579820245
20 2.919828101925886
20 2.281184166007767
20 1.815909051450053
20 1.4736685998874794
20 1.217418446986849
20 1.0246110594583948
20 0.8777164491769808
20 0.7654787320455935
20 0.6789798151104769
20 0.6121495330286217
20 0.5602254544663523
20 0.519755051701807
20 0.48810544722032034
20 0.4632424536924893
20 0.44367394728373893
20 0.42817389421862123
20 0.41588293266723414
20 0.4060511813103891
20 0.39817732482838475
20 0.3917982176691324
20 0.3866556689781995
20 0.3823672008188991
20 0.37883935817117304
20 0.3758146286071315
20 0.3732281876196606
20 0.36783692045388505
20 0.36560724696775193
20 0.3636219542596666
20 0.3604330724196632
20 0.357431358870269
20 0.35567269371400284
20 0.35060204806326783
20 0.34810156290979616
20 0.3459987698669435
20 0.34341761631936746
20 0.34092662278816416
20 0.3384

In [ ]:
molec = "R32"
molec_data_dict = {molec:r32_class} #, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
# all_gp_dict = {"R14" : {list(all_gp_dict["R14"].keys())[0]: list(all_gp_dict["R14"].values())[0]}}
all_gp_dict = {molec : all_gp_dict[molec]}
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, save_data)
ls_results = at_optimizer.optimize_ats()
ls_results

In [ ]:
molec_data_dict = {"R32":r32_class}#, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
setup = opt_atom_types.Problem_Setup(molec_data_dict, all_gp_dict, at_class, save_data)
visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, True, save_data)
best_set = np.array([2.89631543, 4.0, 1.5, 3.1945792, 2.23150945,  3.07915865, 75.0, 75.0, 9.9999988, 29.9003147, 50.0, 50.0])
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, save_data)
theta_guess= setup.values_pref_to_real(best_set)
obj = at_optimizer.calc_obj(theta_guess, True)
print(obj[0])

In [ ]:
molec_data_dict = {"R32":r32_class}#, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
# all_gp_dict = {"R14" : {list(all_gp_dict["R14"].keys())[0]: list(all_gp_dict["R14"].values())[0]}}
# all_gp_dict = {molec : all_gp_dict[molec]}
setup = opt_atom_types.Problem_Setup(molec_data_dict, all_gp_dict, at_class, save_data)
visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, True, save_data)
best_set = np.array([2.89631543, 4.0, 1.5, 3.1945792, 2.23150945,  3.07915865, 75.0, 75.0, 9.9999988, 29.9003147, 50.0, 50.0])
visual.plot_obj_hms(best_set)

In [ ]:
# Determine the index of the row where column 'B' has the minimum value
min_index = ls_results['Min Obj Cum.'].idxmin()

# Retrieve the row corresponding to the minimum value in column 'B'
min_row = ls_results.iloc[min_index]
print(min_row)

In [ ]:
# ls_results.to_csv("test1.csv")

In [ ]:
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
driver = opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, save_data)